In [ ]:
basedir = '/home/abhinavgupta0110/NeuralODEs_ROM_Closure'

import os

is_google_colab = False
is_use_GPU = False

### Mount the Google drive if needed

In [ ]:
if is_use_GPU:
    gpu_info = !nvidia-smi
    gpu_info = '\n'.join(gpu_info)
    if gpu_info.find('failed') >= 0:
        print('No GPU found!')
    else:
        print(gpu_info)

if is_google_colab:
    from google.colab import drive
    drive.mount('/content/drive')

    %pip install quadpy
    
os.chdir(os.path.join(basedir, 'neuralDDE_ROM_Closure'))

### Load modules

In [ ]:
from src.utilities.DDE_Solver import ddeinttf 
import src.solvers.neuralDistDDE_with_adjoint as nddde
import src.advec_diff_case.advec_diff_eqn as adeq

import time
import sys
from IPython.core.debugger import set_trace

import numpy as np
import scipy as spy
import tensorflow as tf
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from shutil import move
import pickle

tf.keras.backend.set_floatx('float32')
import logging
tf.get_logger().setLevel(logging.ERROR)

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

## Define some useful classes

### Class for user-defined arguments

In [ ]:
class res_ad_eq_nDistDDE_args(nddde.nddde_arguments, adeq.ad_eqn_args):

    def __init__(self, batch_time = 12, batch_time_skip = 2, batch_size = 5, epochs = 500, learning_rate = 0.05, decay_rate = 0.95, test_freq = 1, plot_freq = 2, 
                 d_max = 1.1, nn_d1 = 0., nn_d2 = 0.2, state_dim = 2, adj_data_size = 2,
                 model_dir = 'ROM_nDistDDE_testcase/model_dir_test', restart = 0, val_percentage = 0.2,
                 T = 2., nt = 200, L = 1., nx_high_res = 100, nx_low_res = 25, Re = 250, u_bc_0 = 0., u_bc_L =0., isplot = True, reg_coeff = 0.): # add more arguments as needed
        
        nddde.nddde_arguments.__init__(self, data_size = nt, batch_time = batch_time, batch_time_skip = batch_time_skip, batch_size = batch_size, epochs = epochs,
                           learning_rate = learning_rate, decay_rate = decay_rate, test_freq = test_freq, plot_freq = plot_freq, d_max = d_max, nn_d1 = nn_d1, 
                           nn_d2 = nn_d2, state_dim = nx_low_res, adj_data_size = nx_low_res, model_dir = model_dir, restart = restart, val_percentage = val_percentage, isplot = isplot)

        adeq.ad_eqn_args.__init__(self, T = T, nt = nt, L = L, nx = nx_low_res, Re = Re, u_bc_0 = u_bc_0, u_bc_L = u_bc_L,
                              multi_solve_size = batch_size)
        
        self.args_for_high_res = adeq.ad_eqn_args(T = T, nt = nt, L = L, nx = nx_high_res, Re = Re, u_bc_0 = u_bc_0, u_bc_L = u_bc_L, 
                              multi_solve_size = 1)
        
        self.args_for_low_res = adeq.ad_eqn_args(T = T, nt = nt, L = L, nx = nx_low_res, Re = Re, u_bc_0 = u_bc_0, u_bc_L = u_bc_L, 
                              multi_solve_size = 1)
        
        self.nx_high_res = nx_high_res
        self.nx_low_res = nx_low_res
        self.reg_coeff = reg_coeff

### Define the neural net architecture

In [ ]:
class DDEFuncMain(tf.keras.Model):

    def __init__(self, **kwargs):
        super(DDEFuncMain, self).__init__(**kwargs)
        
        self.x1 = tf.keras.layers.Dense(10, activation='tanh',
                                       kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.1), use_bias=True)
        
        self.x2 = tf.keras.layers.Dense(10, activation='tanh',
                                       kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.1), use_bias=True)
        
        self.out = tf.keras.layers.Dense(args.state_dim, activation='linear',
                                       kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.1), use_bias=True)

    def call(self, z):
        for i in range(len(self.layers)):
            z = self.layers[i](z)
        return z

In [ ]:
class DDEFuncAux(tf.keras.Model):

    def __init__(self, **kwargs):
        super(DDEFuncAux, self).__init__(**kwargs)
        
        self.x1 = tf.keras.layers.Dense(5, activation='tanh',
                                       kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.1), use_bias=True)
        
        self.x2 = tf.keras.layers.Dense(5, activation='tanh',
                                       kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.1), use_bias=True)

        self.out = tf.keras.layers.Dense(3, activation='linear',
                                       kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.1), use_bias=True)

    def call(self, z):
        for i in range(len(self.layers)):
            z = self.layers[i](z)
        return z

In [ ]:
class split_zy:
    def __init__(self, zy, args):
        self.zy = zy
        self.args = args

    def __call__(self, t):
        return self.zy(t)[:, :self.args.state_dim]

In [ ]:
class DistDDEFunc:

    def __init__(self, main, aux, rom_model, args):
        self.main = main
        self.aux = aux
        self.rom_model = rom_model
        self.args = args

    def __call__(self, y, t ,d):
        
        get_z = split_zy(y, self.args)

        dz_dt = self.main(y(t)) + self.rom_model(get_z, t) + args.reg_coeff * tf.random.normal([y(t).shape[0], args.state_dim])
        gz_t1 = self.aux(y(t - d[0])[:, :self.args.state_dim])
        gz_t2 = self.aux(y(t - d[1])[:, :self.args.state_dim])
        dy_dt = gz_t1 - gz_t2
        return tf.concat([dz_dt, dy_dt], axis=-1)

In [ ]:
def interp_high_res_to_low_res(u_high_res, x_high_res, x_low_res, t):

    f = spy.interpolate.interp2d(x_high_res, t, tf.squeeze(u_high_res, axis=1).numpy(), kind='cubic')

    u_interp = tf.expand_dims(tf.convert_to_tensor(f(x_low_res, t), dtype=tf.float32), axis=1)

    return u_interp

### Define a custom loss function

In [ ]:
class custom_loss(tf.keras.losses.Loss):

    def call(self, true_y, pred_y):
        loss = tf.reduce_mean(tf.sqrt(tf.reduce_sum(tf.math.squared_difference(pred_y, true_y), axis=-1)), axis=0)
        return loss

### Define a custom plotting function

In [ ]:
class custom_plot:

    def __init__(self, true_y, y_no_nn, x, t, figsave_dir, args):
        self.true_y = true_y
        self.y_no_nn = y_no_nn
        self.t = t
        self.figsave_dir = figsave_dir
        self.args = args
        self.X, self.T = np.meshgrid(x.numpy(), t.numpy())
        self.x = x

    def plot(self, *pred_y, epoch = 0):
        fig = plt.figure(figsize=(24, 4), facecolor='white')
        ax_u_lowres = fig.add_subplot(141, projection='3d')
        ax_u_diff = fig.add_subplot(142, projection='3d')
        ax_u_nn = fig.add_subplot(143, projection='3d')
        ax_u_diff_nn = fig.add_subplot(144, projection='3d')

        ax_u_lowres.cla()
        ax_u_lowres.set_title('AD Eqn Low-Res Solution')
        ax_u_lowres.set_xlabel('x')
        ax_u_lowres.set_ylabel('t')
        ax_u_lowres.plot_surface(self.X, self.T, tf.squeeze(self.y_no_nn, axis=1).numpy(), cmap=cm.coolwarm,
                            linewidth=0, antialiased=False)
        ax_u_lowres.set_xlim(self.x[0], self.x[-1])
        ax_u_lowres.set_ylim(self.t[0], self.t[-1])

        ax_u_diff.cla()
        ax_u_diff.set_title('AD Eqn Difference without NN')
        ax_u_diff.set_xlabel('x')
        ax_u_diff.set_ylabel('t')
        ax_u_diff.plot_surface(self.X, self.T, tf.squeeze(self.true_y - self.y_no_nn, axis=1).numpy(), cmap=cm.coolwarm,
                            linewidth=0, antialiased=False)
        ax_u_diff.set_xlim(self.x[0], self.x[-1])
        ax_u_diff.set_ylim(self.t[0], self.t[-1])

        if epoch != 0 or self.args.restart == 1 :
            ax_u_nn.cla()
            ax_u_nn.set_title('AD Eqn with NN Low-Res Solution')
            ax_u_nn.set_xlabel('x')
            ax_u_nn.set_ylabel('t')
            ax_u_nn.plot_surface(self.X, self.T, tf.squeeze(pred_y[0][:, :, 0:self.args.state_dim], axis=1).numpy(), cmap=cm.coolwarm,
                                linewidth=0, antialiased=False)
            ax_u_nn.set_xlim(self.x[0], self.x[-1])
            ax_u_nn.set_ylim(self.t[0], self.t[-1])

            ax_u_diff_nn.cla()
            ax_u_diff_nn.set_title('AD Eqn Difference with NN')
            ax_u_diff_nn.set_xlabel('x')
            ax_u_diff_nn.set_ylabel('t')
            ax_u_diff_nn.plot_surface(self.X, self.T, tf.squeeze(self.true_y - pred_y[0][:, :, 0:self.args.state_dim], axis=1).numpy(), cmap=cm.coolwarm,
                                linewidth=0, antialiased=False)
            ax_u_diff_nn.set_xlim(self.x[0], self.x[-1])
            ax_u_diff_nn.set_ylim(self.t[0], self.t[-1])

        plt.show() 

        if epoch != 0: 
            fig.savefig(os.path.join(self.figsave_dir, 'img'+str(epoch)))

### Initial Conditions

In [ ]:
class initial_cond:

    def __init__(self, x, app):
        self.x = x
        self.app = app

    def __call__(self, t):
        u0 = self.x / (1. + np.sqrt(1./self.app.t0) * np.exp(self.app.Re * self.x**2 / 4., dtype = np.float64))
        return tf.convert_to_tensor([u0], dtype=tf.float32)

### Initialize model related parameters

In [ ]:
args = res_ad_eq_nDistDDE_args(batch_time = 6, batch_time_skip = 2, batch_size = 8, epochs = 150, learning_rate = 0.075, decay_rate = 0.97, test_freq = 1, plot_freq = 1,
                    d_max = 0.15, nn_d1 = 0., nn_d2 = 0.075, model_dir = 'Res_nDistDDE_testcase/model_dir_case', restart = 0, val_percentage = 1.,
                    T = 1.25, nt = 125, L = 1., nx_high_res = 100, nx_low_res = 25, Re = 1000, u_bc_0 = 0., u_bc_L =0., reg_coeff = 0.)

### Make a copy of the current script

In [ ]:
os.chdir(basedir)

if not os.path.exists(args.model_dir):
    os.makedirs(args.model_dir)

checkpoint_dir_main = os.path.join(args.model_dir, "ckpt_main")
checkpoint_dir_aux = os.path.join(args.model_dir, "ckpt_aux")
checkpoint_prefix_main = os.path.join(checkpoint_dir_main, "cp-{epoch:04d}.ckpt")
checkpoint_prefix_aux = os.path.join(checkpoint_dir_aux, "cp-{epoch:04d}.ckpt")
if not os.path.exists(checkpoint_dir_main):
  os.makedirs(checkpoint_dir_main)
if not os.path.exists(checkpoint_dir_aux):
  os.makedirs(checkpoint_dir_aux)

figsave_dir = os.path.join(args.model_dir, "img")
if not os.path.exists(figsave_dir):
    os.makedirs(figsave_dir)

!jupyter nbconvert --to python neuralDDE_ROM_Closure/testcases/AD_Eqn_Res/neuralDistDDE_Res_AD_Eqn_TestCase.ipynb
move("neuralDDE_ROM_Closure/testcases/AD_Eqn_Res/neuralDistDDE_Res_AD_Eqn_TestCase.py", os.path.join(args.model_dir, "orig_run_file.py"))

with open(os.path.join(args.model_dir, 'args.pkl'), 'wb') as output:
    pickle.dump(args, output, pickle.HIGHEST_PROTOCOL)

### Solve for the high resolution model

In [ ]:
x_high_res = tf.linspace(0., args.L, args.nx_high_res)
t = tf.linspace(0., args.T, args.nt) # Time array

u0 = initial_cond(x_high_res, args.args_for_high_res) # Initial conditions

op = adeq.operators(args.args_for_high_res)

u_high_res = ddeinttf(adeq.ad_eqn(op), u0, t)

# Compute FOM for the validation time
dt = t[1] - t[0]
val_t_len =  args.val_percentage * (t[-1] - t[0])
n_val = np.ceil(np.abs(val_t_len/dt.numpy())).astype(int)
val_t = tf.linspace(t[-1], t[-1] + val_t_len, n_val)

val_u_high_res = ddeinttf(adeq.ad_eqn(op), nddde.create_interpolator(u_high_res, t), val_t)

print('High resolution model done!')

### Solve for low resolution model

In [ ]:
x_low_res = tf.linspace(0., args.L, args.nx_low_res)

u0 = initial_cond(x_low_res, args.args_for_low_res) # Initial conditions

op = adeq.operators(args.args_for_low_res)

u_low_res = ddeinttf(adeq.ad_eqn(op), u0, t)

val_u_low_res = ddeinttf(adeq.ad_eqn(op), nddde.create_interpolator(u_low_res, t), val_t)

print('Low resolution model done!')

In [ ]:
# Interpolate high resolution solution on low resolution grid

true_u_low_res = interp_high_res_to_low_res(u_high_res, x_high_res, x_low_res, t)

val_true_u_low_res = interp_high_res_to_low_res(val_u_high_res, x_high_res, x_low_res, val_t)


## Main part starts here

### Make objects and define learning-rate schedule

In [ ]:
time_meter = nddde.RunningAverageMeter(0.97)

func_main = DDEFuncMain()
func_aux = DDEFuncAux()
func = DistDDEFunc(func_main, func_aux, adeq.ad_eqn(op = adeq.operators(args)), args)
adj_func = nddde.nddde_adj_eqn(func, args)
get_batch = nddde.create_batch(true_u_low_res, u0, t, args)
loss_obj = custom_loss()
plot_obj = custom_plot(tf.concat([true_u_low_res, val_true_u_low_res], axis=0), 
                       tf.concat([u_low_res, val_u_low_res], axis=0), x_low_res, tf.concat([t, val_t], axis=0), figsave_dir, args)
loss_history = nddde.history(args)
val_obj = nddde.create_validation_set_nddde(u0, t, args)

initial_learning_rate = args.learning_rate
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=args.niters,
    decay_rate=args.decay_rate,
    staircase=True)

### Quick test to see how the true coefficients looks like

In [ ]:
if args.restart == 1: 
    func_main.load_weights(tf.train.latest_checkpoint(checkpoint_dir_main))
    func_aux.load_weights(tf.train.latest_checkpoint(checkpoint_dir_aux))
    process_true_z0 = nddde.process_DistDDE_IC(u0, func_aux, t_lowerlim = t[0] - args.nn_d2, t_upperlim = t[0] - args.nn_d1)
    pred_zy = ddeinttf(func, process_true_z0, tf.concat([t, val_t], axis=0), fargs=([args.nn_d1, args.nn_d2],))
    
    plot_obj.plot(pred_zy[:, :, :args.state_dim], epoch = 0)

    loss_history.read()
    
    initial_learning_rate = 0.05
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate, decay_steps=args.niters, decay_rate=0.95, staircase=True)
    
else:
    plot_obj.plot(epoch = 0)

### Training starts here

In [ ]:
optimizer_main = tf.keras.optimizers.RMSprop(learning_rate = lr_schedule)
optimizer_aux = tf.keras.optimizers.RMSprop(learning_rate = lr_schedule)

nDistDDE_train_obj = nddde.train_nDistDDE(func = func, adj_func = adj_func, d = [args.nn_d1, args.nn_d2], loss_obj = loss_obj, batch_obj = get_batch,
                            checkpoint_dir_aux = checkpoint_prefix_aux, optimizer_main = optimizer_main, optimizer_aux = optimizer_aux, args = args, plot_obj = plot_obj, time_meter = time_meter, checkpoint_dir_main = checkpoint_prefix_main,
                            validation_obj = val_obj, loss_history_obj = loss_history)

nDistDDE_train_obj.train(true_u_low_res, u0, t, val_true_u_low_res)